In [15]:
# Install the required libraries
# !pip install folium

In [17]:
# Import the libraries
import pandas as pd
import folium
import requests
from datetime import datetime

In [19]:
# URL for significant earthquakes in the past 30 days from the USGS
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson'

print("Fetching live earthquake data from USGS...")

try:
    # Fetch the data
    response = requests.get(url)
    response.raise_for_status() # Check if the request was successful
    data = response.json()

    # Extract the relevant information from the GeoJSON structure
    earthquake_list = []
    for feature in data['features']:
        properties = feature['properties']
        coords = feature['geometry']['coordinates']
        earthquake_list.append({
            'Place': properties['place'],
            'Magnitude': properties['mag'],
            'Time': datetime.fromtimestamp(properties['time'] / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            'Longitude': coords[0],
            'Latitude': coords[1]
        })

    # Create a DataFrame
    df = pd.DataFrame(earthquake_list)
    print(f"Successfully loaded {len(df)} significant earthquake records.")
    display(df.head())

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")

Fetching live earthquake data from USGS...
Successfully loaded 16 significant earthquake records.


,Place,Magnitude,Time,Longitude,Latitude
0,"194 km WNW of Abepura, Indonesia",6.5,2025-10-16 06:48:55,138.9382,-2.1676
1,Drake Passage,6.3,2025-10-16 02:42:32,-61.6217,-59.9066
2,"2 km SSE of Tambongon, Philippines",5.7,2025-10-12 18:05:59,123.9375,10.9457
3,Drake Passage,7.6,2025-10-10 21:29:20,-61.7990,-60.1958
4,"23 km ESE of Santiago, Philippines",6.7,2025-10-10 12:12:07,126.7553,7.1716


In [21]:
# Create a map centered on the world
world_map = folium.Map(location=[0, 0], zoom_start=2)

print("Adding earthquake data points to the map...")

# Iterate through each earthquake and add a circle marker
for index, row in df.iterrows():
    # Define the popup text
    popup_text = f"<b>Magnitude: {row['Magnitude']}</b><br>Location: {row['Place']}<br>Time: {row['Time']}"
    
    # Add a circle marker
    # The radius is based on the magnitude to make stronger quakes appear larger
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Magnitude'] * 2.5, # Multiply magnitude to make circles more visible
        color='red',
        fill=True,
        fill_color='darkred',
        fill_opacity=0.6,
        popup=popup_text
    ).add_to(world_map)

print("Map creation complete!")

Adding earthquake data points to the map...
Map creation complete!


In [23]:
# Display the map
world_map

***Live data from the USGS API was used to plot significant earthquakes from the last 30 days on an interactive world map. The visualization effectively used magnitude to scale the size of each data point, providing an immediate and intuitive understanding of where the most powerful seismic events occurred globally.***